# Imports

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook



# Functions

# Create Credentials and API Object

In [2]:
# Load API Credentials
with open('/Users/Foreground/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)



## Set search terms and file paths

In [3]:
# set our API call parameters 
LOCATION = 'Seattle, WA'
TERM = 'Pizza'


In [4]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_WA_pizza.json"
JSON_FILE



'Data/results_in_progress_WA_pizza.json'

In [5]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")


[i] Data/results_in_progress_WA_pizza.json not found. Saving empty list to file.


In [6]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')



- 0 previous results found.


# Functions

In [7]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  



# Making the first call to get the first page of data

In [8]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()



dict_keys(['businesses', 'total', 'region'])

In [9]:
## How many results total?
total_results = results['total']
total_results



2100

In [10]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page



20

In [11]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages



105

In [12]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)


In [13]:
# ## Create a new empty json file (exist the previous if it exists)
# create_json_file(JSON_FILE, delete_if_exists=True)
# ## Load previous results and use len of results for offset
# with open(JSON_FILE,'r') as f:
#     previous_results = json.load(f)
    
# ## set offset based on previous results
# n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages



- 0 previous results found.


105

In [14]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)



  0%|          | 0/105 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


# Final results

In [15]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,-FOAQv22SXtSBs7nptI3UA,serious-pie-downtown-seattle-2,Serious Pie Downtown,https://s3-media1.fl.yelpcdn.com/bphoto/Yge-Xa...,False,https://www.yelp.com/biz/serious-pie-downtown-...,4555,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 47.61304626911818, 'longitude': -...","[pickup, delivery]",$$,"{'address1': '2001 4th Ave', 'address2': None,...",+12068387388,(206) 838-7388,1422.856174
1,IrohtoYjnAR_vc6w6CRCxA,moto-pizza-west-seattle-seattle,MOTO Pizza - West Seattle,https://s3-media2.fl.yelpcdn.com/bphoto/RA8N1B...,False,https://www.yelp.com/biz/moto-pizza-west-seatt...,180,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 47.56207, 'longitude': -122.38509}",[delivery],$$,"{'address1': '4526 42nd Ave SW', 'address2': '...",+12064208880,(206) 420-8880,7965.106751
2,ENxs0vvtkJYxQqaZ3sbEjw,my-friend-dereks-seattle,My Friend Derek's,https://s3-media1.fl.yelpcdn.com/bphoto/MVV6S3...,False,https://www.yelp.com/biz/my-friend-dereks-seat...,32,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...",5.0,"{'latitude': 47.651515, 'longitude': -122.343613}",[],NaN,"{'address1': '3632 Woodland Park Ave N', 'addr...",,,2964.793378
3,1rkgyfwzXG7qSdVZdfRt7g,roccos-seattle,Rocco's,https://s3-media1.fl.yelpcdn.com/bphoto/7h9z8m...,False,https://www.yelp.com/biz/roccos-seattle?adjust...,1805,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 47.6145745, 'longitude': -122.345...","[pickup, delivery]",$$,"{'address1': '2312 2nd Ave', 'address2': '', '...",+12063974210,(206) 397-4210,1438.882536
4,JaK1jA7D3Z4_8g9DySLZ8A,west-of-chicago-pizza-company-seattle,West of Chicago Pizza Company,https://s3-media2.fl.yelpcdn.com/bphoto/hzCo4v...,False,https://www.yelp.com/biz/west-of-chicago-pizza...,193,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 47.56123, 'longitude': -122.37985}",[delivery],$$,"{'address1': '3770 SW Alaska St', 'address2': ...",+12063393337,(206) 339-3337,7864.921259


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,l6dwcgNyn-ROwguZd-46fA,chuan-bbq-seattle-7,Chuan BBQ,https://s3-media1.fl.yelpcdn.com/bphoto/UCSlZW...,False,https://www.yelp.com/biz/chuan-bbq-seattle-7?a...,160,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",3.5,"{'latitude': 47.68527, 'longitude': -122.34412}","[delivery, pickup]",$$,"{'address1': '7714 Aurora Ave N', 'address2': ...",+12064540389,(206) 454-0389,6688.648830
996,qwQskljAH0XEuPh1Bcs8rw,pizza-hut-normandy-park,Pizza Hut,https://s3-media1.fl.yelpcdn.com/bphoto/kAdxt1...,False,https://www.yelp.com/biz/pizza-hut-normandy-pa...,8,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",2.5,"{'latitude': 47.44240504315486, 'longitude': -...","[delivery, pickup]",NaN,"{'address1': '17833 1st Ave S', 'address2': 'S...",+12062444900,(206) 244-4900,20350.208770
997,XL6UtF0GljGKOIG0FLum3g,pcc-community-markets-redmond-redmond,PCC Community Markets - Redmond,https://s3-media2.fl.yelpcdn.com/bphoto/J9FA6k...,False,https://www.yelp.com/biz/pcc-community-markets...,139,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",4.5,"{'latitude': 47.7016, 'longitude': -122.09339}",[],$$,"{'address1': '11435 Avondale Rd NE', 'address2...",+14252851400,(425) 285-1400,20002.231986
998,Kyr_35FK6pi9L9Ohp8FXPQ,pizza-hut-renton,Pizza Hut,https://s3-media4.fl.yelpcdn.com/bphoto/8oThqA...,False,https://www.yelp.com/biz/pizza-hut-renton?adju...,81,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.0,"{'latitude': 47.4988145, 'longitude': -122.181...","[delivery, pickup]",$,"{'address1': '2705 Sunset Blvd NE', 'address2'...",+14252284800,(425) 228-4800,18222.418967
999,6p9QZkPYxf8APQ1inQZ_GQ,round-table-pizza-silverdale,Round Table Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/pobAaV...,False,https://www.yelp.com/biz/round-table-pizza-sil...,91,"[{'alias': 'pizza', 'title': 'Pizza'}]",3.0,"{'latitude': 47.656651, 'longitude': -122.695006}","[delivery, pickup]",$$,"{'address1': '3276 NW Plaza Rd', 'address2': '...",+13606984040,(360) 698-4040,27156.401932


In [16]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()



0

## Save the file

In [17]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_WA_pizza.csv.gz', compression='gzip',index=False)

